<a href="https://colab.research.google.com/github/dongjaeseo/CS2705/blob/main/lung_cancer_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
import bisect

In [ ]:
class mask_image():
    def __init__(self, image):
        self.image = image
        self.boundary = self.find_boundary()
        self.cg = self.centroid()
        self.theta_dict = self.theta()
        self.distance_theta = self.find_dist()

    def centroid(self):
        row_sum = 0
        column_sum = 0
        count = 0
        for i, row in enumerate(self.image):
            for j, pixel in enumerate(row):
                if pixel == 1:
                    row_sum += i
                    column_sum += j
                    count += 1

        if count == 0:
            raise TypeError("Wrong image input: Only masked image of 0 and 1 is allowed")

        return column_sum/count, row_sum/count

    def find_boundary(self):
        boundary = []
        for i, row in enumerate(self.image):
            for j, pixel in enumerate(row):

                if i != 0:
                    if self.image[i-1][j] < pixel:
                        boundary.append((j,i-0.5))
                
                if j != len(self.image[0])-1:
                    if self.image[i][j+1] < pixel:
                        boundary.append((j+0.5, i))
                
                if i != len(self.image)-1:
                    if self.image[i+1][j] < pixel:
                        boundary.append((j,i+0.5))

                if j != 0:
                    if self.image[i][j-1] < pixel:
                        boundary.append((j-0.5, i))

        return boundary
    
    # Represent angle between [0, 2pi]
    def theta(self):
        theta_boundary = {}
        for bound in self.boundary:
            theta = np.arctan((self.cg[1]-bound[1])/(bound[0]-self.cg[0]))
            # in 3rd and 4th quadrant // arctan (-pi/2, pi/2)
            if bound[0] < self.cg[0]:
                theta += np.pi
            elif bound[1] > self.cg[1]:
                theta += 2*np.pi
            theta_boundary.update({theta: bound})
        
        theta_boundary = dict(sorted(theta_boundary.items(), key=lambda item: item[0]))

        return theta_boundary
    
    def find_dist(self):
        distance_theta = {}
        for key, value in self.theta_dict.items():
            dist = np.sqrt(np.square(value[0]-self.cg[0]) + np.square(value[1]-self.cg[1]))
            distance_theta[key] = dist
        
        return distance_theta

    def plot(self):
        plt.imshow(self.image, cmap = 'gray')
        for i in self.boundary:
            plt.plot(i[0], i[1], marker = "o", markersize = 3, markeredgecolor = "red", markerfacecolor = "green")
            plt.plot(self.cg[0], self.cg[1], marker = "o", markersize = 1, markeredgecolor = "red", markerfacecolor = "green")
        plt.show()
    
    def plot_uniform(self, angle_degree):
        plt.imshow(self.image, cmap = 'gray')
        for key, value in self.theta_distance(angle_degree).items():
            plt.plot(self.cg[0]+value*np.cos(key), self.cg[1]-value*np.sin(key), marker = "o", markersize = 3, markeredgecolor = "red", markerfacecolor = "green")
            plt.plot(self.cg[0], self.cg[1], marker = "o", markersize = 1, markeredgecolor = "red", markerfacecolor = "green")
        plt.show()

    def theta_distance(self, angle_degree):
        num = 360//angle_degree
        theta_dist = {}
        for i in range(num):
            angle_rad = np.pi / 180. * i * angle_degree
            
            if self.distance_theta.get(angle_rad):
                res = self.distance_theta.get(angle_rad)
            else:
                keys = list(self.distance_theta.keys())
                self.distance_theta[min(keys)+ 2*np.pi] = self.distance_theta[min(keys)]
                self.distance_theta[max(keys)- 2*np.pi] = self.distance_theta[max(keys)]
                self.distance_theta = dict(sorted(self.distance_theta.items(), key=lambda item: item[0]))
                
                nx_theta = min([i for i in list(self.distance_theta.keys()) if i>angle_rad])
                pv_theta = max([i for i in list(self.distance_theta.keys()) if i<angle_rad])
                
                next = self.distance_theta[nx_theta]
                prev = self.distance_theta[pv_theta]

                # Calculating the position of angle_rad between next and prev
                slope = (prev*np.sin(pv_theta) - next*np.sin(nx_theta)) / (prev*np.cos(pv_theta) - next*np.cos(nx_theta))
                temp = slope - np.tan(angle_rad)
                temp = 1/temp
                temp *= (prev*np.cos(pv_theta)*np.tan(angle_rad) - prev*np.sin(pv_theta))
                temp += prev*np.cos(pv_theta)
                res = temp / np.cos(angle_rad)

            theta_dist[angle_rad] = res
        
        return theta_dist

In [ ]:
image1 = cv2.imread('/content/drive/MyDrive/project/mask2.png', 0)
image1 = image1/ 255.
image1 = np.where(image1 < 0.5, 0, 1)

image1 = mask_image(image1)
print(image1.theta_distance(60))
image1.plot_uniform(8)

In [ ]:
image1 = cv2.imread('/content/drive/MyDrive/project/mask3.png', 0)
image1 = image1/ 255.
image1 = np.where(image1 < 0.5, 0, 1)

image1 = mask_image(image1)
print(image1.theta_distance(60))
image1.plot_uniform(2)

In [ ]:
image1 = cv2.imread('/content/drive/MyDrive/project/mask2.png', 0)
image1 = image1/ 255.
image1 = np.where(image1 < 0.5, 0, 1)

image1 = mask_image(image1)
distance = image1.find_dist()

theta_dict = image1.theta()

print('distance: ')
print(distance)

theta_dict = image1.theta()
print('\ntheta - coordinate:')
print(theta_dict)

print('\ncentroid: ', image1.centroid())
image1.plot()

In [ ]:
distance_theta = image1.find_dist()
distance_theta

In [ ]:
image1 = cv2.imread('/content/drive/MyDrive/project/mask1.png', 0)
image1 = image1/ 255.
image1 = np.where(image1 < 0.5, 0, 1)

image1 = mask_image(image1)
distance = image1.find_dist()

theta_dict = image1.theta()

print('distance: ')
print(distance)

theta_dict = image1.theta()
print('\ntheta - coordinate:')
print(theta_dict)

print('\ncentroid: ', image1.centroid())
image1.plot()

distance_theta = image1.find_dist()
distance_theta

In [ ]:
image1 = cv2.imread('/content/drive/MyDrive/project/mask3.png', 0)
image1 = image1/ 255.
image1 = np.where(image1 < 0.5, 0, 1)

image1 = mask_image(image1)
distance = image1.find_dist()

theta_dict = image1.theta()

print('distance: ')
print(distance)

theta_dict = image1.theta()
print('\ntheta - coordinate:')
print(theta_dict)

print('\ncentroid: ', image1.centroid())
image1.plot()

distance_theta = image1.find_dist()
distance_theta